# OpenCV Level 2

In [51]:
import numpy as np
import cv2

## Threshold
Lets look at an image first....

In [16]:
image = cv2.imread('bookpage.jpg')
cv2.imshow("Book image",image)
cv2.waitKey()
cv2.destroyAllWindows()

Wasn't easy to read, was it?
<br>
The reason for that are the dark areas, which are, region with lower pixel values.....<br>
<br>
<b>but</b>, if we can find the dark pixels and replace it bigger values.....

In [21]:
ret,thresh = cv2.threshold(image,12,255,cv2.THRESH_BINARY) #Thresh is the generated image and ret is confirmation
#threshold = 12
#maxval = 255
#Type of threshold = BINARY
cv2.imshow("Threshold",thresh)
cv2.waitKey()
cv2.destroyAllWindows()

That makes it significantly more readable.....but how?
<br>
We set the threshold as 12, which are the almost black pixels and selected the BINARY threshold. The duo turned every pixel bellow the threshold to 0 and above the threshold to maxval i.e. 255
<br>
<br>
<b>But the color metrices still makes it somewhat difficult to read the text.</b> <br>So maybe we can try a grayscaled image......

In [22]:
grayscale = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
ret,thresh = cv2.threshold(grayscale,12,255,cv2.THRESH_BINARY) #Thresh is the generated image and ret is confirmation
#threshold = 12
#maxval = 2555
cv2.imshow("Threshold",thresh)
cv2.waitKey()
cv2.destroyAllWindows()

Well that did not work.......................but on a brighter side, it is obvious why...<br><br>
Thresholding simply turned the darker area to completely black, making the middel part unreadable.<br>This happened because the lighting in the image is not uniform and therefore, keeping the same threshold value for the entire image is not ideal......
<br><br><b>So for the solution,</b>
<br>If we could take different threshold values for different areas in the image, it might work!........and the adaptiveThreshold() function does exactly that......

In [30]:
grayscale = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
thresh = cv2.adaptiveThreshold(grayscale,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,115,1) 
#maxval = 255
#adaptive type = MEAN
#Area size = 115
#Constant to add = 1
cv2.imshow("Threshold",thresh)
cv2.waitKey()
cv2.destroyAllWindows()

### A lot better isn't it.....
The MEAN function decided the threshold by calculating the mean around the area and thus, the threshold of every grid was different.<br>
Another adaptive thresholding method is ADAPTIVE_THRESH_GAUSSIAN_C, which selects the value from a gaussian distribution for the surrounding area......try it out!
<br><br>
Now, for the other thresholding methods......<br>
<img src="Notes_image/threshold_types.jpg">
<br><br>
There you go......

## Mask
<br>
<b>What is a mask?</b><br>
It is basically an image with only zeros and 255s and can be used to remove the unwanted areas.
<br>
This can be made using the threshold function on the grayscaled image.....

In [44]:
image = cv2.imread('py.jpg')
#Converting the image to grayscale
grayscale = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
#Making the mask
ret,mask = cv2.threshold(grayscale,220,255,cv2.THRESH_BINARY_INV)
cv2.imshow("Mask",mask)
cv2.waitKey()
cv2.destroyAllWindows()

The black areas are the one that will be removed and the white ones are kept in the image.<br><br>
<b>But....how do we use it?.....</b><br>
Using the bitwise AND operation.<br>
Now, simple logic: If we AND an image with itself, we will get the same image. Now when we use a mask on the result, it removes the black areas......

In [47]:
masked_image = cv2.bitwise_and(image,image,mask=mask) #Applying the mask
cv2.imshow("Masked",masked_image)
cv2.waitKey()
cv2.destroyAllWindows()
#See what it did there......

## Filters
#### Behold......lot of concepts.......
The notion is simple, keep only those colors that we want........
<br>If the object is<i> Amethyst </i>in color (try finding one....), then remove all the colors except those which are in that range.....Simple?
<br> But pulling this off with BGR or RGB images is not that feasible.....why?
<br><br>
<table><tr>
<td><img src="Notes_image/Bright.jpg",height="62" width="62"></td>
<td><img src="Notes_image/Dark.jpg",height="62" width="62"></td>
</table></tr>
Look at these two images.<br>
Due to the difference in brightness levels, value of pixels in the red matrix do not remain the same. This makes it difficult to decide the range that is to be kept with the filter.
<br><br>
<b>The sollution</b>,<br>
The HSV color mode. It stands for Hew(or color), Saturation(or intensity) and Value(or brightness) which makes it clear why this one is better. It saperates the lighting details form the color details making the filter work better under different lighting conditions.
<img src="Notes_image/HSV.png">
<br><b> Now, Refer this graph to decide the color range. X axis the hew and Y axis is the Saturation</b>
<br>
The range will be 2 lists indicating the upper and lower values of the color. This will be fed to the inRange() function and guess the outcome.......its a mask!
<br>
But that will be a little noisey. To handle that, we can use some blurring and erosion to get rid of that extra stuff.....
<br><br>
<b> Now pick up an object and lets being......</b>

In [54]:
cap = cv2.VideoCapture(0)

while True:
    #Recording the video
    _,frame = cap.read()
    #Making the hsv image
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    #The color range for that red spool up there.....
    lower_red = np.array([0,100,20])
    upper_red = np.array([10,255,255])
    #Generating the mask
    mask = cv2.inRange(hsv,lower_red,upper_red)
    #Some erossion and dialation
    mask = cv2.erode(mask,None,iterations=2)
    
    #Removing the removed part
    res = cv2.bitwise_and(frame,frame,mask=mask)
    
    #Eliminating that noisey stuff....
    blur = cv2.medianBlur(res,15)

    #Lets see what we have got here
    cv2.imshow('blur',blur)
    cv2.imshow('mask',mask)
    cv2.imshow("frame",frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):  
        break
        
        
    #Press q to exit from camera
    
cap.release()  #Release the camera (necessary)
cv2.destroyAllWindows()

<b>And....done</b>

## Contours

Basically, lines that connect regions with the same color.....
<br>
Lets try this on a simple image......but a grayscaled one.....

In [90]:
image = cv2.imread('py.jpg')
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
contours,_ = cv2.findContours(gray,cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

In [91]:
contours

[array([[[  0,   0]],
 
        [[  0, 629]],
 
        [[629, 629]],
 
        [[629,   0]]], dtype=int32), array([[[351, 574]],
 
        [[352, 573]],
 
        [[353, 573]],
 
        [[354, 574]],
 
        [[353, 575]],
 
        [[352, 575]]], dtype=int32), array([[[348, 574]],
 
        [[349, 573]],
 
        [[350, 573]],
 
        [[351, 574]],
 
        [[350, 575]],
 
        [[349, 575]]], dtype=int32), array([[[344, 574]],
 
        [[345, 573]],
 
        [[346, 574]],
 
        [[346, 575]],
 
        [[345, 576]],
 
        [[344, 575]]], dtype=int32), array([[[279, 574]],
 
        [[280, 573]],
 
        [[282, 573]],
 
        [[283, 574]],
 
        [[282, 575]],
 
        [[280, 575]]], dtype=int32), array([[[350, 573]],
 
        [[351, 572]],
 
        [[352, 573]],
 
        [[351, 574]]], dtype=int32), array([[[274, 572]],
 
        [[275, 571]],
 
        [[277, 573]],
 
        [[276, 574]],
 
        [[273, 574]],
 
        [[272, 573]],
 
        [[273, 5

After printing the 'contours' variable, what do think you'll see.<br>
Let me spoil it for you, it a collections of arrays. And those arrays have points that are to be connected to make one of the many contours.
<br>Lets draw 'em....

In [92]:
cv2.drawContours(image,contours,-1,(255,0,0),3)
# The third arg is for the index. -1 means all of them......
cv2.imshow("Contours",image)
cv2.waitKey()
cv2.destroyAllWindows()

### Features of contours.....
The first one....moments.
<br> Basically, 3 points that can be used to tell many things about the image......
<br>Example (m01/m00, m10/m00) is the center of that image....see....easy! 

In [93]:
#Lets pick up the largest one 
cnt = max(contours[1:],key = cv2.contourArea)

In [94]:
cv2.contourArea(cnt)

192670.5

<b> Wondering which one it is...?

In [95]:
image = cv2.imread('py.jpg')
cv2.drawContours(image,[c],-1,(255,0,0),3)
# The third arg is for the index. -1 means all of them......
cv2.imshow("Contours",image)
cv2.waitKey()
cv2.destroyAllWindows()

<b>That one.......